# Raster processing with XArray

http://xarray.pydata.org

Robin Wilson &nbsp; &nbsp; &nbsp; @sciremotesense &nbsp; &nbsp; &nbsp; robin@rtwilson.com

**UPDATE THIS**

http://bit.do/xarraytalk &nbsp; &nbsp; &nbsp; https://github.com/robintw/xarraytalk

## Problem

Processing large time series of raster data is **difficult**

**Example**:

We have many decades of daily raster data, and want to get:

- Seasonal means and standard deviations

- Long time-series across specific points

- Individual images at specific times

# HOW?

## `XArray`

The power of `pandas` for multidimension arrays

- Labelled

- Multidimensional

- Efficient

- Easy to use!

## Related tools / Prerequisites?

- `python` (obviously!)
- `numpy`
- `pandas`
- `matplotlib`
- `GDAL`
- `rasterio`

## Previous experience?

**Q:** How many people are experienced with `numpy`?

**Q:** ...with `pandas`?

**Q:** ...with `GDAL`?

In [ ]:
import datetime

%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [26]:
import xarray as xr

## Example

In [ ]:
PM25 = xr.open_dataarray('/Users/robin/code/MAIACProcessing/All2014.nc')

In [ ]:
PM25.shape

In [ ]:
PM25.dims

In [ ]:
seasonal = PM25.groupby('time.season').mean(dim='time')

In [ ]:
seasonal.plot.imshow(col='season', robust=True)

In [ ]:
time_series = PM25.isel(x=1000, y=1100).to_pandas().dropna()

In [ ]:
time_series

In [ ]:
one_day = PM25.sel(time='2014-02-20')

In [ ]:
one_day.plot(robust=True)

## Plan

- Introduction to XArray

- Efficient processing with `dask` and `dask.distributed`

- Getting raster data into XArray

- Geographic processing

- Where next...?

## Introduction to XArray

`xarray.DataArray` is a fancy, labelled version of a `numpy.ndarray`

`xarray.Dataset` is a collection of multiple `DataArray`s which share dimensions

In [ ]:
arr = np.random.rand(3, 4, 2)

In [ ]:
xr.DataArray(arr)

In [ ]:
xr.DataArray(arr, dims=('x', 'y', 'time'))

In [ ]:
da = xr.DataArray(arr,
                  dims=('x', 'y', 'time'),
                  coords={'x': [10, 20, 30],
                          'y': [0.3, 0.7, 1.3, 1.5],
                          'time': [datetime.datetime(2016, 3, 5),
                                   datetime.datetime(2016, 4, 7)]})

In [ ]:
da

In [ ]:
da.sel(time='2016-03-05')

In [ ]:
da.isel(time=1)

In [ ]:
da.sel(x=slice(0, 20))

In [ ]:
da.mean(dim='time')

In [ ]:
da.mean(dim=['x', 'y'])

In [ ]:
PM25.sel(time='2014').groupby('time.month').std(dim='time')

## Efficient processing with `dask` and `dask.distributed`

`dask` creates a *computational graph* of your processing steps, and then executes it *as efficiently as possible*.

This includes *only loading data that is actually needed* and *only processing things once*.

## NetCDF
- Data format for multidimensional array data

- Basically the same as a `Dataset`

- Can read in *chunks*

In [ ]:
data = xr.open_mfdataset(['DaskTest1.nc', 'DaskTest2.nc'], chunks={'time':10})['data']
avg = data.mean(dim='time')

[Dask execution graph](ExampleGraph_1.png)

In [ ]:
seasonal = data.groupby('time.season').mean(dim='time')

[Dask execution graph](ExampleGraph_2.png)

## `dask.distributed`

All of these different *chunks*, and separate processing chains can be run on **separate computers**.

- [Dask Distributed Example](Dask Distributed Example.ipynb)
- [Live Dashboard](http://localhost:8787)

## Getting raster data into `xarray`

`xarray` can read various raster formats directly:

- NetCDF
- HDF
- GRIB

However, `xarray` can't _currently_ read other standard raster formats like:

- GeoTIFF
- Erdas IMAGINE
- Arc Grids
- ENVI format
- etc...

## `rasterio`

A nice, Pythonic interface to `GDAL` - making it really easy to read almost any raster file into Python

In [ ]:
import rasterio

with rasterio.open('SPOT_ROI.tif') as src:
    data = src.read(1)
    
print(data)

## Joining them up
All we need to do is write some functions to read from `rasterio` and create a `DataArray`

But it's a bit more difficult than that...

- Need to deal with geographic metadata
- Geographic co-ordinates
- Dimension names

In [4]:
from rasterio_to_xarray import rasterio_to_xarray, xarray_to_rasterio, xarray_to_rasterio_by_band

In [3]:
data = rasterio_to_xarray('SPOT_ROI.tif')
data

<xarray.DataArray (y: 1644, x: 1435)>
array([[ 51.,  51.,  51., ...,  32.,  32.,  29.],
       [ 51.,  51.,  51., ...,  29.,  30.,  27.],
       [ 54.,  54.,  52., ...,  30.,  29.,  28.],
       ..., 
       [ 33.,  34.,  34., ...,  31.,  35.,  35.],
       [ 34.,  34.,  34., ...,  32.,  34.,  34.],
       [ 34.,  34.,  34., ...,  31.,  33.,  38.]])
Coordinates:
  * y        (y) float64 1.484e+05 1.484e+05 1.484e+05 1.484e+05 1.484e+05 ...
  * x        (x) float64 4.301e+05 4.301e+05 4.302e+05 4.302e+05 4.302e+05 ...
Attributes:
    crs: +ellps=airy +k=0.999601 +lat_0=49 +lon_0=-2 +no_defs +proj=tmerc +towgs84=446.448,-125.157,542.06,0.15,0.247,0.842,-20.489 +units=m +x_0=400000 +y_0=-100000
    affine: (430130.2396, 10.0, 0.0, 148415.8755, 0.0, -10.0)

## Example - combining multiple files into a time-series

Take a large number of MAIAC files (satellite-measured PM2.5 grids), one from each orbit of a satellite, and put them into one large array with a time dimension.

In [27]:
def maiac_file_to_da(filename):
    da = rasterio_to_xarray(filename)
    
    time_str = os.path.basename(filename)[17:-17]
    time_obj = datetime.datetime.strptime(time_str, '%Y%j%H%M')
    da.coords['time'] = time_obj
    
    return da

In [28]:
import glob, os, datetime

In [29]:
files = glob.glob('MAIAC_files/*.tif')
files

['MAIAC_files/MAIACAAOT.h00v01.20031061440.hdf_projPM25.tif',
 'MAIAC_files/MAIACTAOT.h00v01.20020781120.hdf_projPM25.tif',
 'MAIAC_files/MAIACTAOT.h00v01.20020781300.hdf_projPM25.tif',
 'MAIAC_files/MAIACTAOT.h00v01.20020781435.hdf_projPM25.tif',
 'MAIAC_files/MAIACTAOT.h00v01.20020871115.hdf_projPM25.tif',
 'MAIAC_files/MAIACTAOT.h00v01.20020871255.hdf_projPM25.tif',
 'MAIAC_files/MAIACTAOT.h00v01.20020871430.hdf_projPM25.tif',
 'MAIAC_files/MAIACTAOT.h00v01.20020881020.hdf_projPM25.tif',
 'MAIAC_files/MAIACTAOT.h00v01.20020881200.hdf_projPM25.tif',
 'MAIAC_files/MAIACTAOT.h00v01.20020881335.hdf_projPM25.tif']

In [31]:
all_das = [maiac_file_to_da(filename) for filename in files]

In [33]:
MAIAC_combined = xr.concat(all_das, dim='time')

In [34]:
MAIAC_combined

<xarray.DataArray (time: 10, y: 1162, x: 1240)>
array([[[ nan,  nan, ...,  nan,  nan],
        [ nan,  nan, ...,  nan,  nan],
        ..., 
        [ nan,  nan, ...,  nan,  nan],
        [ nan,  nan, ...,  nan,  nan]],

       [[ nan,  nan, ...,  nan,  nan],
        [ nan,  nan, ...,  nan,  nan],
        ..., 
        [ nan,  nan, ...,  nan,  nan],
        [ nan,  nan, ...,  nan,  nan]],

       ..., 
       [[ nan,  nan, ...,  nan,  nan],
        [ nan,  nan, ...,  nan,  nan],
        ..., 
        [ nan,  nan, ...,  nan,  nan],
        [ nan,  nan, ...,  nan,  nan]],

       [[ nan,  nan, ...,  nan,  nan],
        [ nan,  nan, ...,  nan,  nan],
        ..., 
        [ nan,  nan, ...,  nan,  nan],
        [ nan,  nan, ...,  nan,  nan]]], dtype=float32)
Coordinates:
  * y        (y) float64 1.429e+06 1.428e+06 1.427e+06 1.426e+06 1.424e+06 ...
  * x        (x) float64 -9.476e+05 -9.464e+05 -9.451e+05 -9.439e+05 ...
  * time     (time) datetime64[ns] 2003-04-16T14:40:00 2002-03-19T11:20

## Getting raster data out of xarray

In [43]:
mean = MAIAC_combined.mean(dim='time', keep_attrs=True)

In [44]:
xarray_to_rasterio(mean, 'Mean.tif')

In [45]:
daily = MAIAC_combined.groupby('time.day').mean(dim='time')

In [48]:
xarray_to_rasterio_by_band(daily, 'Daily_', dim='day')

Exported 16
Exported 19
Exported 28
Exported 29


In [51]:
!ls Daily_*.tif

Daily_16.tif Daily_19.tif Daily_28.tif Daily_29.tif


## Geographic processing

## Where next...?

- `xarray` is under rapid development
- General approach is stable, some details may change
- Very responsive developers

- Raster I/O will be built-in to `xarray` soon-ish

- `dask` is rapidly improving too

## Resources

- These slides are available online at **TODO**
- The notebooks are available at **TODO**